In [121]:
import numpy as np
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px
import plotly.graph_objs as go
# ML libraries - idk which one to use yet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing, tree
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import cKDTree

In [3]:
# NASA API
def get_nasa_power_data(lat, lon, start_date, end_date):
    """
    Fetches NASA POWER API data for given latitude, longitude, and time range.

    Args:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - start_date (str): Start date in YYYYMMDD format.
    - end_date (str): End date in YYYYMMDD format.

    Returns:
    - Pandas DataFrame with selected weather parameters.
    """

    # Specify multiple parameters in the API request
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M"

    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)

    data = response.json()

    # Convert JSON response to DataFrame and transpose it
    nasa_weather = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T

    # Reset index and rename date column
    nasa_weather.reset_index(inplace=True)
    nasa_weather.rename(columns={"index": "date"}, inplace=True)

    # Convert date column to proper datetime format
    nasa_weather["date"] = pd.to_datetime(nasa_weather["date"], format="%Y%m%d", errors="coerce")
    nasa_weather.dropna(subset=["date"], inplace=True)  # Remove invalid date rows

    nasa_weather.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m"
    }, inplace=True)
    
    # Add Rounded_Lat and Rounded_Lng for merging
    nasa_weather['Rounded_Lat'] = lat
    nasa_weather['Rounded_Lng'] = lon
    
    # Display DataFrame
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    display(nasa_weather)  # Works in Jupyter Notebook

    return nasa_weather

In [4]:
# Example: Fetch data for different locations
df_la = get_nasa_power_data(34.05, -118.25, "20240101", "20240107")  # Los Angeles


 Weather Data for Latitude 34.05, Longitude -118.25



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng
0,2024-01-01,0.0,6.12,0.04,12.14,1.64,34.05,-118.25
1,2024-01-02,0.0,7.05,0.09,11.55,1.59,34.05,-118.25
2,2024-01-03,0.0,6.75,5.87,10.86,3.25,34.05,-118.25
3,2024-01-04,0.0,2.14,0.02,9.76,2.89,34.05,-118.25
4,2024-01-05,0.0,1.99,0.00,10.80,2.00,34.05,-118.25
5,2024-01-06,0.0,1.23,0.12,10.50,2.86,34.05,-118.25
6,2024-01-07,0.0,-0.09,0.26,7.77,5.29,34.05,-118.25


In [5]:
# inspect us_accident data
us_accidents = pd.read_csv('US_Accidents_March23.csv')
us_accidents.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [23]:
unique_values = us_accidents['Weather_Condition'].unique()

print(unique_values)

['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squalls' 'N/A Precipitation' 'Cloudy / Windy' 'Light Fog' 'Sand'
 'Snow G

In [24]:
# list of all columns in US Accident Data
drop_cols = ['ID',
            'Source',
            # 'Severity', # Severity = target column, 1-4, where 1 indicates the least impact on traffic
            'Start_Time',
            'End_Time',
            'Start_Lat',  
            'Start_Lng', 
            'End_Lat',
            'End_Lng',
            'Distance(mi)', # Distance(mi) = target column?, length of road extent affected by accident in miles
            'Description', # Description = human description of accident
            'Street', 
            'City', 
            'County',
            'State',
            'Zipcode',
            'Country',
            'Timezone',
            'Airport_Code',
            'Weather_Timestamp', # Weather_Timestamp = shows time-stamp of weather observation record (in local time)
            # 'Temperature(F)',
            'Wind_Chill(F)',
            'Humidity(%)',
            'Pressure(in)',
            'Visibility(mi)',
            'Wind_Direction',
            # 'Wind_Speed(mph)',
            # 'Precipitation(in)',
            # 'Weather_Condition',
            'Amenity',
            'Bump',
            'Crossing',
            'Give_Way',
            'Junction',
            'No_Exit',
            'Railway',
            'Roundabout',
            'Station',
            'Stop',
            'Traffic_Calming',
            'Traffic_Signal',
            'Turning_Loop',
            'Sunrise_Sunset', # day or night based on sunrise/sunset
            'Civil_Twilight', # day or night based on civil twilight
            'Nautical_Twilight', # day or night based on nautical twilight
            'Astronomical_Twilight'] # day or night based on astronomical twilight

In [28]:
# string feature that if kept will need to be encoded for ML
str_features = 'Weather_Condition'

In [37]:
def prepare_data(df, split, ml_keep=[]):
    """
    Prepares the US Accidents DataFrame for merging with NASA weather data, keeping only necessary columns.
    
    Args:
    - df (DataFrame): Raw US Accidents dataset.
    - split (boolean): if true, split df by target and predictor data
    - ml_drop (list): other columns to drop for machine learning model (adjustable if we decide a variable is not good at predicting)
    
    Returns:
    - DataFrame: Processed DataFrame with 'date', 'Rounded_Lat', 'Rounded_Lng', and 'Severity' columns.
    """
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert time columns to datetime format
    df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

    # Remove rows with invalid 'Start_Time' values
    df = df[df['Start_Time'].notnull()].copy()

    # Extract 'date' from 'Start_Time' for merging with NASA weather data
    df['date'] = df['Start_Time'].dt.date

    # Filter for coordinates within LA County
    # could use 'County' to filter
    # df = df[df['County'] == 'Los Angeles']
    df = df[(df['Start_Lat'].between(33.7, 34.8)) & (df['Start_Lng'].between(-119.0, -117.6))]

    # Round latitude and longitude to 2 decimal places for approximate matching
    df['Rounded_Lat'] = df['Start_Lat'].round(2)
    df['Rounded_Lng'] = df['Start_Lng'].round(2)

    # Keep only relevant columns for merging
    # df = df[['date', 'Rounded_Lat', 'Rounded_Lng', 'Severity']]
    df = df.drop(drop_cols, axis=1)
    
    # encode str_features
    le = preprocessing.LabelEncoder()
    df['Weather_Condition'] = le.fit_transform(df['Weather_Condition'])
    
    # drop NaN
    df = df.dropna()
    
    if split:
        X=df[ml_keep]
        y=df['Severity']
        return X,y

    return df

In [107]:
# inspect would-be cleaned data
us_accidents_2 = prepare_data(us_accidents, split=False)
us_accidents_2
median_value = us_accidents_2[us_accidents_2['Wind_Speed(mph)'] != 0]['Wind_Speed(mph)'].median()
mean_value2 = us_accidents_2[us_accidents_2['Precipitation(in)'] != 0]['Precipitation(in)'].mean()
median_value2 = us_accidents_2[us_accidents_2['Precipitation(in)'] != 0]['Precipitation(in)'].median()
max_value2 = us_accidents_2['Precipitation(in)'].max()
us_accidents_2['Weather_Total'] = us_accidents_2['Precipitation(in)'] + (us_accidents_2['Wind_Speed(mph)']/100)
us_accidents_2
#Wind speed: max = 58, mean = 7.316, median = 6.0
#Precipitation: max = 2.05, mean = 0.07588, median = 0.04
#Divide Wind speed by 100 to get them to similar "weight" in the overall weather condition calculation

,Severity,Temperature(F),Wind_Speed(mph),Precipitation(in),Weather_Condition,date,Rounded_Lat,Rounded_Lng,Weather_Total
42863,2,87.0,8.0,0.0,7,2016-06-21,34.27,-118.64,0.08
42879,3,87.0,8.0,0.0,7,2016-06-21,34.34,-118.51,0.08
42970,2,94.0,9.0,0.0,7,2016-06-21,34.45,-118.58,0.09
43012,3,87.0,10.0,0.0,7,2016-06-21,34.35,-118.54,0.10
43016,2,85.0,10.0,0.0,7,2016-06-21,34.40,-118.45,0.10
...,...,...,...,...,...,...,...,...,...
7728382,2,79.0,5.0,0.0,7,2019-08-23,33.85,-117.84,0.05
7728384,2,78.0,6.0,0.0,7,2019-08-23,34.06,-118.00,0.06
7728387,2,75.0,9.0,0.0,7,2019-08-23,33.74,-117.84,0.09
7728391,2,73.0,10.0,0.0,37,2019-08-23,33.78,-117.85,0.10


In [108]:
# Normalize the data
df_accident = us_accidents_2.copy()

# Normalize the data (only 'Weather_Total' column)
scaler = StandardScaler()
X = df_accident[['Weather_Total']]  # Use only the 'Weather_Total' column
X_scaled = scaler.fit_transform(X)

# Apply K-means clustering
kmeans = KMeans(n_clusters=10, random_state=42)
df_accident['weather_cluster'] = kmeans.fit_predict(X_scaled)

# Inverse transform cluster centers for interpretation
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
print("Cluster Centers:\n", cluster_centers)

# Assign severity levels to clusters
# severity_levels = ["Clear", "Fair", "Moderate", "Unsettled", "Poor", "Harsh", "Turbulent", "Severe", "Extreme", "Critical"]
# df_accident['severity_level'] = df_accident['cluster'].map(lambda x: severity_levels[x])

print(df_accident.head())

/opt/anaconda3/envs/PIC16B-25W/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster Centers:
 [[9.46987504e-02]
 [3.00898414e-02]
 [3.51977607e-01]
 [5.46050236e-05]
 [1.74051231e-01]
 [5.44640207e-02]
 [6.06422764e-01]
 [1.30692440e-01]
 [7.46444360e-02]
 [2.37943677e-01]]
       Severity  Temperature(F)  Wind_Speed(mph)  Precipitation(in)  \
42863         2            87.0              8.0                0.0   
42879         3            87.0              8.0                0.0   
42970         2            94.0              9.0                0.0   
43012         3            87.0             10.0                0.0   
43016         2            85.0             10.0                0.0   

       Weather_Condition        date  Rounded_Lat  Rounded_Lng  Weather_Total  \
42863                  7  2016-06-21        34.27      -118.64           0.08   
42879                  7  2016-06-21        34.34      -118.51           0.08   
42970                  7  2016-06-21        34.45      -118.58           0.09   
43012                  7  2016-06-21        34.35 

In [127]:
df_accident['coordinates'] = df_accident['Rounded_Lat'].astype(str) + "_" + df_accident['Rounded_Lng'].astype(str)

def estimate_risk(latitude, longitude, target_cluster, df, kmeans, scaler):
    # Find all accidents at the given location
    location_accidents = df[(df['Rounded_Lat'] == latitude) & (df['Rounded_Lng'] == longitude)]
    
    if location_accidents.empty:
        return None  # No accidents at this location
    
    # Get the cluster center for the target weather condition
    target_center = scaler.inverse_transform(kmeans.cluster_centers_[target_cluster].reshape(1, -1))
    
    # Calculate distances to the target cluster center
    distances = []
    for _, row in location_accidents.iterrows():
        # Get 'Weather_Total' for the accident
        weather_feature = row[['Weather_Total']].values.reshape(1, -1)
        # Normalize the feature
        weather_feature_scaled = scaler.transform(pd.DataFrame(weather_feature, columns=['Weather_Total']))
        # Calculate distance to the target cluster center
        distance = euclidean_distances(weather_feature_scaled, target_center)
        distances.append(distance[0][0])
    
    if len(distances) > 1:
        min_distance = min(distances)
        max_distance = max(distances)
        normalized_distances = [(d - min_distance) / (max_distance - min_distance) for d in distances]
    else:
        normalized_distances = [0.5]  # Default score if only one accident exists
    
    # Return the average normalized distance as the risk score
    risk_score = 1 - np.mean(normalized_distances)  # Invert so higher score = higher risk
    return risk_score

# Example usage
latitude = 34.05  # Example latitude (Los Angeles)
longitude = -118.25  # Example longitude (Los Angeles)
target_cluster = 0 # Example target weather cluster (e.g., "Severe Weather")
risk = estimate_risk(latitude, longitude, target_cluster, df_accident, kmeans, scaler)
print(f"Estimated Risk: {risk}")

Estimated Risk: 0.31070623860568247


In [143]:
"""
Differences to above:
The risk score is based on how close past accidents are to the input weather cluster center. 
Accidents closer to the input cluster contribute more to the risk score.
Distances are normalized to a 0-1 range
If no accidents occurred at the location, the risk score is 0.
The score reflects how much the input weather condition deviates from the "normal" conditions at that location.
"""
def estimate_risk(latitude, longitude, target_cluster, df, kmeans, scaler):
    # Find all accidents at the given location
    location_accidents = df[(df['Rounded_Lat'] == latitude) & (df['Rounded_Lng'] == longitude)]
    
    if location_accidents.empty:
        return 0  # No accidents at this location, risk is 0
    
    # Get the cluster center for the target weather condition
    target_center = kmeans.cluster_centers_[target_cluster].reshape(1, -1)
    
    # Calculate distances of all accidents to the target cluster center
    distances = []
    for _, row in location_accidents.iterrows():
        weather_feature = row[['Weather_Total']].values.reshape(1, -1)
        weather_feature_scaled = scaler.transform(pd.DataFrame(weather_feature, columns=['Weather_Total']))
        distance = euclidean_distances(weather_feature_scaled, target_center)
        distances.append(distance[0][0])
    
    # Normalize distances to a 0-1 range
    max_distance = max(distances) if max(distances) != 0 else 1
    normalized_distances = [d / max_distance for d in distances]
    
    # Calculate risk score as the average of (1 - normalized distances)
    risk_score = np.mean([1 - d for d in normalized_distances])
    
    return risk_score

latitude = 34.05  # Example latitude (Los Angeles)
longitude = -118.25  # Example longitude (Los Angeles)
target_cluster = 1 # Example target weather cluster (e.g., "Severe Weather")
risk = estimate_risk(latitude, longitude, target_cluster, df_accident, kmeans, scaler)
print(f"Estimated Risk: {risk}")

Estimated Risk: 0.7340143178724018
